# Data Preprocessing

This file contains the preprocessing for the players visualizations

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("data/player_valuations.csv")  # Replace with your file

# Convert 'date' to datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")

df["player_id"] = df["player_id"].astype(int)

# Remove missing dates
df = df.dropna(subset=["date"])

# Fill missing market values using forward fill per player
df["market_value_in_eur"] = df.groupby("player_id")["market_value_in_eur"].fillna(method="ffill")

# Convert market value to millions
df["market_value_in_millions"] = df["market_value_in_eur"] / 1_000_000

# Sort by player and date
df = df.sort_values(by=["player_id", "date"])

# Save processed data
df.to_csv("processed_data/player_valuations.csv", index=False)



#################################################################################
# Create a summary of the data for each player get # of red cards, yellow cards, assists, goals, and minutes played

import pandas as pd

base_path = "./data/"
appearances = pd.read_csv(base_path + "appearances.csv")
events = pd.read_csv(base_path + "game_events.csv")
players = pd.read_csv(base_path + "players.csv")

# Normalize event types
events["type_norm"] = events["type"].astype(str).str.strip().str.lower()

# 🥅 Goals
goals = events[(events["type_norm"] == "goals") & (events["player_id"].notnull())]
goal_counts = goals.groupby("player_id").size().reset_index(name="goals")

# 🎯 Assists
assists = events[events["player_assist_id"].notnull()]
assist_counts = assists.groupby("player_assist_id").size().reset_index(name="assists")
assist_counts.rename(columns={"player_assist_id": "player_id"}, inplace=True)

# ⏱️ Minutes (90 per appearance)
appearances["estimated_minutes"] = 90
minutes = appearances.groupby("player_id")["estimated_minutes"].sum().reset_index(name="minutes")

# 🟨🟥 Cards
card_counts = appearances.groupby("player_id")[["yellow_cards", "red_cards"]].sum().reset_index()

# 🧠 Merge all
summary = pd.merge(goal_counts, assist_counts, on="player_id", how="outer")
summary = pd.merge(summary, minutes, on="player_id", how="outer")
summary = pd.merge(summary, card_counts, on="player_id", how="outer")
summary = pd.merge(summary, players[["player_id", "name"]], on="player_id", how="left")

# Fill missing and convert to int
summary.fillna(0, inplace=True)
summary[["goals", "assists", "yellow_cards", "red_cards", "minutes"]] = summary[
    ["goals", "assists", "yellow_cards", "red_cards", "minutes"]
].astype(int)

# Final structure
summary = summary[["player_id", "name", "goals", "assists", "yellow_cards", "red_cards", "minutes"]]
summary.to_csv("player_summary.csv", index=False)
print("✅ Saved to player_summary.csv")

# 🏆 Print Top 15 per category
def print_top_15(column, label):
    print(f"\n🏆 Top 15 Players by {label}:\n")
    top = summary.sort_values(column, ascending=False).head(15)
    print(top[["name", column]].to_string(index=False))
